In [ ]:
from pydrake.all import *
from manipulation.station import LoadScenario, MakeHardwareStation
from manipulation.meshcat_utils import AddMeshcatTriad
from pydrake.geometry import Rgba, Cylinder
from pydrake.systems.sensors import RgbdSensor, CameraConfig
from pathlib import Path
import numpy as np
import tempfile
import os
import sys

sys.path.append(str(Path.cwd()))
from throw_helpers import create_q_knots, joint_angles_to_pose

print("Imports loaded")

Imports loaded


In [3]:
# Start meshcat
meshcat = StartMeshcat()
print(f"Meshcat: {meshcat.web_url()}")

INFO:drake:Meshcat listening for connections at http://localhost:7001


Meshcat: http://localhost:7001


In [4]:
# Perception helper functions

def depth_image_to_point_cloud(depth_image, rgb_image, camera_info):
    """Convert depth image to colored point cloud in camera frame"""
    height, width = depth_image.shape
    fx = camera_info.focal_x()
    fy = camera_info.focal_y()
    cx = camera_info.center_x()
    cy = camera_info.center_y()
    
    u = np.arange(width)
    v = np.arange(height)
    u_grid, v_grid = np.meshgrid(u, v)
    
    valid = (depth_image > 0.01) & (depth_image < 5.0)
    
    u_valid = u_grid[valid]
    v_valid = v_grid[valid]
    z_valid = depth_image[valid]
    
    x = (u_valid - cx) * z_valid / fx
    y = (v_valid - cy) * z_valid / fy
    z = z_valid
    
    points = np.column_stack([x, y, z])
    colors = rgb_image[valid].astype(float) / 255.0
    
    return points, colors


def transform_points_to_world(points, X_CameraToWorld):
    """Transform points from camera frame to world frame"""
    points_world = []
    for pt in points:
        pt_world = X_CameraToWorld @ pt
        points_world.append(pt_world)
    return np.array(points_world)


def make_camera_transform(camera_pos, look_at_pos):
    """Create camera transform that looks from camera_pos to look_at_pos"""
    forward = look_at_pos - camera_pos
    forward = forward / np.linalg.norm(forward)
    world_up = np.array([0, 0, 1])
    right = np.cross(forward, world_up)
    right = right / np.linalg.norm(right)
    down = np.cross(forward, right)
    R_WorldToCamera = RotationMatrix(np.column_stack([right, down, forward]))
    return RigidTransform(R_WorldToCamera, camera_pos)


print("Perception functions defined")

Perception functions defined


In [21]:
# Scene setup: Robot + Extended Table + 2 Balls + 4 Cameras + 2 Bins (Red & Blue)

# Create temporary SDF files for balls
temp_dir = tempfile.mkdtemp()

red_ball_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="red_ball">
    <link name="ball">
      <inertial>
        <mass>0.1</mass>
        <inertia>
          <ixx>0.0001</ixx><ixy>0</ixy><ixz>0</ixz>
          <iyy>0.0001</iyy><iyz>0</iyz><izz>0.0001</izz>
        </inertia>
      </inertial>
      <visual name="visual">
        <geometry><sphere><radius>0.05</radius></sphere></geometry>
        <material><diffuse>0.9 0.1 0.1 1.0</diffuse></material>
      </visual>
      <collision name="collision">
        <geometry><sphere><radius>0.05</radius></sphere></geometry>
      </collision>
    </link>
  </model>
</sdf>
"""

blue_ball_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="blue_ball">
    <link name="ball">
      <inertial>
        <mass>0.1</mass>
        <inertia>
          <ixx>0.0001</ixx><ixy>0</ixy><ixz>0</ixz>
          <iyy>0.0001</iyy><iyz>0</iyz><izz>0.0001</izz>
        </inertia>
      </inertial>
      <visual name="visual">
        <geometry><sphere><radius>0.05</radius></sphere></geometry>
        <material><diffuse>0.2 0.4 0.9 1.0</diffuse></material>
      </visual>
      <collision name="collision">
        <geometry><sphere><radius>0.05</radius></sphere></geometry>
      </collision>
    </link>
  </model>
</sdf>
"""

red_ball_path = os.path.join(temp_dir, "red_ball.sdf")
blue_ball_path = os.path.join(temp_dir, "blue_ball.sdf")

with open(red_ball_path, 'w') as f:
    f.write(red_ball_sdf)
with open(blue_ball_path, 'w') as f:
    f.write(blue_ball_sdf)

# Ball positions on table
ball_radius = 0.05
red_pos = np.array([0.0, -0.5, ball_radius])
blue_pos = np.array([-0.15, -0.55, ball_radius])

# Asset paths
abs_table_sdf_path = f"{Path.cwd()}/assets/table_extended.sdf"
abs_bin_red_sdf_path = f"{Path.cwd()}/assets/bin_red.sdf"
abs_bin_blue_sdf_path = f"{Path.cwd()}/assets/bin_blue.sdf"

# Bin positions - side by side at same Y distance (moved slightly closer)
BIN_CENTER_Y = 1.65  # Moved from 1.7 to 1.65
BIN_OFFSET_X = 0.35
red_bin_pos = np.array([BIN_OFFSET_X, BIN_CENTER_Y, 0.0])
blue_bin_pos = np.array([-BIN_OFFSET_X, BIN_CENTER_Y, 0.0])

TABLE_CENTER_Y = 1.0

# Scenario YAML with free-body balls and two bins
scenario_yaml = f"""
directives:
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
    default_joint_positions:
      iiwa_joint_1: [-1.57]
      iiwa_joint_2: [0.1]
      iiwa_joint_3: [0]
      iiwa_joint_4: [-1.2]
      iiwa_joint_5: [0]
      iiwa_joint_6: [1.6]
      iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: wsg
    file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
      translation: [0, 0, 0.09]
      rotation: !Rpy {{ deg: [90, 0, 90] }}
- add_model:
    name: table
    file: file://{abs_table_sdf_path}
- add_weld:
    parent: world
    child: table::table_link
    X_PC:
      translation: [0.0, {TABLE_CENTER_Y}, -0.05]
      rotation: !Rpy {{ deg: [0, 0, 0] }}
- add_model:
    name: bin_red
    file: file://{abs_bin_red_sdf_path}
- add_weld:
    parent: world
    child: bin_red::bin_link
    X_PC:
      translation: [{red_bin_pos[0]}, {red_bin_pos[1]}, 0.0]
- add_model:
    name: bin_blue
    file: file://{abs_bin_blue_sdf_path}
- add_weld:
    parent: world
    child: bin_blue::bin_link
    X_PC:
      translation: [{blue_bin_pos[0]}, {blue_bin_pos[1]}, 0.0]
- add_model:
    name: red_ball
    file: file://{red_ball_path}
    default_free_body_pose:
      ball:
        translation: [{red_pos[0]}, {red_pos[1]}, {red_pos[2]}]
- add_model:
    name: blue_ball
    file: file://{blue_ball_path}
    default_free_body_pose:
      ball:
        translation: [{blue_pos[0]}, {blue_pos[1]}, {blue_pos[2]}]

model_drivers:
    iiwa: !IiwaDriver
      control_mode: position_only
      hand_model_name: wsg
    wsg: !SchunkWsgDriver {{}}
"""

# Build diagram with cameras
meshcat.Delete()
scenario = LoadScenario(data=scenario_yaml)
builder = DiagramBuilder()

station = MakeHardwareStation(scenario, meshcat=meshcat)
builder.AddSystem(station)

plant = station.GetSubsystemByName("plant")
scene_graph = station.GetSubsystemByName("scene_graph")

# Get initial gripper pose BEFORE building diagram
temp_context = station.CreateDefaultContext()
temp_plant_context = plant.GetMyContextFromRoot(temp_context)
X_WG_initial = plant.EvalBodyPoseInWorld(temp_plant_context, plant.GetBodyByName("body"))
print(f"Initial gripper pose: {X_WG_initial.translation()}")

# Add VTK renderer
renderer_name = "my_renderer"
scene_graph.AddRenderer(renderer_name, MakeRenderEngineVtk(RenderEngineVtkParams()))

# Camera config
width, height = 640, 480
camera_config = CameraConfig()
camera_config.width = width
camera_config.height = height
camera_config.fps = 10.0
camera_config.renderer_name = renderer_name

# ORIGINAL 4 camera positions that worked for red ball
workspace_center = np.array([0.0, -0.5, 0.05])
camera_positions = [
    np.array([0.4, -0.15, 0.5]),    # Front-right, HIGH
    np.array([-0.35, -0.9, 0.25]),  # Back-left, LOW
    np.array([-0.5, -0.4, 0.4]),    # Left side, MEDIUM
    np.array([0.5, -0.6, 0.35]),    # Right side
]

camera_transforms = [make_camera_transform(pos, workspace_center) for pos in camera_positions]
camera_sensors = []
camera_infos = []

for i, X_cam in enumerate(camera_transforms):
    color_cam, depth_cam = camera_config.MakeCameras()
    sensor = builder.AddSystem(RgbdSensor(
        parent_id=scene_graph.world_frame_id(),
        X_PB=X_cam,
        color_camera=color_cam,
        depth_camera=depth_cam
    ))
    builder.Connect(station.GetOutputPort("query_object"), sensor.query_object_input_port())
    camera_sensors.append(sensor)
    camera_infos.append(color_cam.core().intrinsics())

# Build diagram
diagram = builder.Build()
context = diagram.CreateDefaultContext()
diagram.ForcedPublish(context)

# Visualize cameras
cam_colors = [
    Rgba(0.0, 0.5, 1.0, 0.8),  # Cyan
    Rgba(1.0, 0.5, 0.0, 0.8),  # Orange
    Rgba(0.8, 0.0, 0.8, 0.8),  # Purple
    Rgba(0.0, 1.0, 0.5, 0.8),  # Teal
]

for i, (X_cam, color) in enumerate(zip(camera_transforms, cam_colors), 1):
    meshcat.SetObject(f"camera{i}/body", Box(0.05, 0.05, 0.08), color)
    meshcat.SetTransform(f"camera{i}/body", X_cam)

print(f"\nScene setup complete:")
print(f"  Extended table: 2m x 5m (centered at y={TABLE_CENTER_Y})")
print(f"  Red ball: {red_pos}")
print(f"  Blue ball: {blue_pos}")
print(f"  Red bin: [{red_bin_pos[0]:.2f}, {red_bin_pos[1]:.2f}, 0.0]")
print(f"  Blue bin: [{blue_bin_pos[0]:.2f}, {blue_bin_pos[1]:.2f}, 0.0]")
print(f"  4 cameras (original positions)")

Initial gripper pose: [ 3.70783219e-04 -4.65616808e-01  6.79321579e-01]

Scene setup complete:
  Extended table: 2m x 5m (centered at y=1.0)
  Red ball: [ 0.   -0.5   0.05]
  Blue ball: [-0.15 -0.55  0.05]
  Red bin: [0.35, 1.65, 0.0]
  Blue bin: [-0.35, 1.65, 0.0]
  4 cameras (original positions)


In [22]:
# PERCEPTION: Multi-camera capture and point cloud fusion (4 CAMERAS)
# Detects BOTH red and blue balls for sorting

print("=== MULTI-CAMERA PERCEPTION PIPELINE (4 cameras) ===\n")

# Lists to store point clouds from all cameras
all_points_world = []
all_colors = []

# Process each camera
for cam_idx, (sensor, cam_info, X_cam) in enumerate(zip(camera_sensors, camera_infos, camera_transforms), 1):
    print(f"--- Camera {cam_idx} ---")
    
    # Get images from this sensor
    sensor_context = sensor.GetMyContextFromRoot(context)
    
    color_image = sensor.color_image_output_port().Eval(sensor_context)
    rgb_img = color_image.data[:, :, :3].astype(np.uint8)
    
    depth_image = sensor.depth_image_32F_output_port().Eval(sensor_context)
    depth_img = depth_image.data.squeeze().astype(np.float32)
    
    # Convert to point cloud in camera frame
    points_camera, colors = depth_image_to_point_cloud(depth_img, rgb_img, cam_info)
    print(f"  Generated {len(points_camera)} points")
    
    # Transform to world frame
    points_world = transform_points_to_world(points_camera, X_cam)
    
    all_points_world.append(points_world)
    all_colors.append(colors)

# Merge all point clouds
merged_points = np.vstack(all_points_world)
merged_colors = np.vstack(all_colors)

print(f"\n✓ Merged {len(merged_points)} points from {len(camera_sensors)} cameras\n")

# BALL DETECTION - Both RED and BLUE
print("--- Ball Detection (Red & Blue) ---")

# Step 1: Find points that are ABOVE the table (z > 0.02)
above_table_mask = merged_points[:, 2] > 0.02
above_table_points = merged_points[above_table_mask]
above_table_colors = merged_colors[above_table_mask]
print(f"Points above table (z > 0.02): {len(above_table_points)}")


def fit_sphere_least_squares(points, known_radius):
    """
    Fit a sphere to surface points using least squares.
    Given points on a sphere surface and known radius, find the center.
    
    Minimizes: sum of (distance_from_center - radius)^2
    """
    from scipy.optimize import least_squares
    
    def residuals(center, points, radius):
        distances = np.linalg.norm(points - center, axis=1)
        return distances - radius
    
    # Initial guess: centroid of points
    initial_center = np.mean(points, axis=0)
    
    result = least_squares(residuals, initial_center, args=(points, known_radius))
    
    return result.x


def detect_ball_by_color(points, colors, color_type):
    """Detect a ball by color using least-squares sphere fitting."""
    if color_type == "red":
        score = colors[:, 0] - np.maximum(colors[:, 1], colors[:, 2])
    else:  # blue
        score = colors[:, 2] - np.maximum(colors[:, 0], colors[:, 1])

    # Get top scoring points
    num_points = int(max(50, int(len(points) * 0.02)))
    threshold = np.sort(score)[-num_points]

    is_colored = score >= threshold
    colored_points = points[is_colored].copy()

    if len(colored_points) < 10:
        return None

    # Initial clustering using median to remove outliers
    centroid = np.median(colored_points, axis=0)

    cluster_radius = 0.06
    for iteration in range(3):
        distances = np.linalg.norm(colored_points - centroid, axis=1)
        inliers = distances < cluster_radius
        if np.sum(inliers) > 10:
            centroid = np.median(colored_points[inliers], axis=0)
            colored_points = colored_points[inliers]
            cluster_radius = 0.05

    # FIT SPHERE to the clustered surface points
    fitted_center = fit_sphere_least_squares(colored_points, ball_radius)
    
    # Force Z to ball_radius (ball sits on table)
    detected_pos = np.array([fitted_center[0], fitted_center[1], ball_radius])

    return detected_pos, colored_points


# Detect RED ball
detected_red_pos = None
if len(above_table_points) > 0:
    result = detect_ball_by_color(above_table_points, above_table_colors, "red")
    if result is not None:
        detected_red_pos, red_cluster = result
        error_red = np.linalg.norm(detected_red_pos - red_pos)
        print(f"\nRED BALL:")
        print(f"  Detected: {detected_red_pos}")
        print(f"  Ground truth: {red_pos}")
        print(f"  Error: {error_red*1000:.1f}mm")
        print(f"  Cluster size: {len(red_cluster)} points")
        
        # Visualize detected red ball (yellow outline)
        meshcat.SetObject("perception/detected_red", Sphere(0.055), Rgba(1.0, 1.0, 0.0, 0.6))
        meshcat.SetTransform("perception/detected_red", RigidTransform(detected_red_pos))
        
        # Plot the red cluster points
        for i, pt in enumerate(red_cluster[::5]):
            meshcat.SetObject(f"perception/red_pts/pt_{i}", Sphere(0.008), Rgba(1.0, 0.0, 0.0, 0.9))
            meshcat.SetTransform(f"perception/red_pts/pt_{i}", RigidTransform(pt))

# Detect BLUE ball
detected_blue_pos = None
if len(above_table_points) > 0:
    result = detect_ball_by_color(above_table_points, above_table_colors, "blue")
    if result is not None:
        detected_blue_pos, blue_cluster = result
        error_blue = np.linalg.norm(detected_blue_pos - blue_pos)
        print(f"\nBLUE BALL:")
        print(f"  Detected: {detected_blue_pos}")
        print(f"  Ground truth: {blue_pos}")
        print(f"  Error: {error_blue*1000:.1f}mm")
        print(f"  Cluster size: {len(blue_cluster)} points")
        
        # Visualize detected blue ball (cyan outline)
        meshcat.SetObject("perception/detected_blue", Sphere(0.055), Rgba(0.0, 1.0, 1.0, 0.6))
        meshcat.SetTransform("perception/detected_blue", RigidTransform(detected_blue_pos))
        
        # Plot the blue cluster points
        for i, pt in enumerate(blue_cluster[::5]):
            meshcat.SetObject(f"perception/blue_pts/pt_{i}", Sphere(0.008), Rgba(0.0, 0.0, 1.0, 0.9))
            meshcat.SetTransform(f"perception/blue_pts/pt_{i}", RigidTransform(pt))

# Summary
print("\n" + "="*60)
if detected_red_pos is not None:
    print(f"✓ DETECTED RED BALL - Error: {error_red*1000:.1f}mm")
else:
    print("✗ RED BALL NOT DETECTED")
    
if detected_blue_pos is not None:
    print(f"✓ DETECTED BLUE BALL - Error: {error_blue*1000:.1f}mm")
else:
    print("✗ BLUE BALL NOT DETECTED")
print("="*60)

=== MULTI-CAMERA PERCEPTION PIPELINE (4 cameras) ===

--- Camera 1 ---
  Generated 299155 points
--- Camera 2 ---
  Generated 250791 points
--- Camera 3 ---
  Generated 287167 points
--- Camera 4 ---
  Generated 271444 points

✓ Merged 1108557 points from 4 cameras

--- Ball Detection (Red & Blue) ---
Points above table (z > 0.02): 86089

RED BALL:
  Detected: [ 2.24477300e-04 -4.99852757e-01  5.00000000e-02]
  Ground truth: [ 0.   -0.5   0.05]
  Error: 0.3mm
  Cluster size: 992 points

BLUE BALL:
  Detected: [-0.14975433 -0.55002004  0.05      ]
  Ground truth: [-0.15 -0.55  0.05]
  Error: 0.2mm
  Cluster size: 2419 points

✓ DETECTED RED BALL - Error: 0.3mm
✓ DETECTED BLUE BALL - Error: 0.2mm


In [23]:
# TRAJECTORY PLANNING: Pick + Throw with Z-rotation for aiming
# Plans trajectory for ONE ball at a time (red first)

if detected_red_pos is None:
    raise RuntimeError("Cannot plan trajectory - red ball not detected")

print("=== TRAJECTORY PLANNING (Red Ball) ===")

# Grasp parameters
GRIPPER_OPEN = 0.107
GRIPPER_CLOSED = 0.0
GRASP_HEIGHT = 0.12
APPROACH_HEIGHT = 0.1

# Throw parameters
RELEASE_FRAC = 0.5
THROW_DURATION = 0.4

# Z-rotation offsets for aiming (hardcoded for red vs blue)
# Negative = rotate clockwise (toward +X, red bin)
# Positive = rotate counterclockwise (toward -X, blue bin)
RED_THROW_ANGLE = np.radians(-15)   # Angle to hit red bin at +X
BLUE_THROW_ANGLE = np.radians(15)   # Angle to hit blue bin at -X

def design_grasp_pose(X_WO: RigidTransform) -> RigidTransform:
    """Grasp pose above object with gripper pointing down."""
    return X_WO @ RigidTransform(RotationMatrix.MakeXRotation(-np.pi/2), [0, 0, GRASP_HEIGHT])


def create_pick_throw_trajectory(ball_pos, throw_angle, ball_name="ball"):
    """
    Create pick and throw trajectory for a ball with specified throw direction.
    """
    print(f"\n--- Planning {ball_name} trajectory ---")
    print(f"  Ball position: {ball_pos}")
    print(f"  Throw angle: {np.degrees(throw_angle):.1f} deg")
    
    # Ball pose from perception
    X_WBall = RigidTransform(p=ball_pos)
    
    # Pick trajectory poses
    X_WG_grasp = design_grasp_pose(X_WBall)
    X_WG_approach = RigidTransform(p=[0, 0, APPROACH_HEIGHT]) @ X_WG_grasp
    
    # Pick timing
    t_approach = 1.0
    t_descend = 0.5
    t_grasp = 0.5
    t_lift = 0.5
    
    t0 = 0
    t1 = t_approach
    t2 = t1 + t_descend
    t3 = t2 + t_grasp
    t4 = t3 + t_lift
    
    # Create pick pose trajectory
    pose_traj = PiecewisePose.MakeLinear(
        [t0, t1, t2, t3, t4],
        [X_WG_initial, X_WG_approach, X_WG_grasp, X_WG_grasp, X_WG_approach]
    )
    
    # Convert pick poses to joint angles
    num_samples = 50
    t_samples = np.linspace(t0, t4, num_samples)
    poses = [pose_traj.GetPose(t) for t in t_samples]
    q_pick_knots = create_q_knots(plant, poses)
    q_pick_traj = PiecewisePolynomial.CubicShapePreserving(t_samples[:len(q_pick_knots)], np.array(q_pick_knots).T)
    
    # Throw trajectory with Z-rotation for aiming
    base_heading = np.pi / 2  # Points in +Y direction
    throw_heading = base_heading + throw_angle
    ja1 = throw_heading - np.pi
    
    PRETHROW_JA = np.array([ja1, 0, 0, 1.9, 0, -1.9, np.pi])
    THROWEND_JA = np.array([ja1, 0, 0, 0.4, 0, -0.4, np.pi])
    
    # Interpolate throw motion
    t_throw_samples = np.linspace(0, THROW_DURATION, 30)
    q_throw_samples = [PRETHROW_JA + (t / THROW_DURATION) * (THROWEND_JA - PRETHROW_JA) for t in t_throw_samples]
    
    # Combine pick + throw trajectories
    T_GO_TO_PRETHROW = 1.5
    t_pick_end = q_pick_traj.end_time()
    
    t_full = list(np.linspace(0, t_pick_end, 50))
    t_full += [t_pick_end + T_GO_TO_PRETHROW]
    t_full += list(t_pick_end + T_GO_TO_PRETHROW + t_throw_samples[1:])
    
    q_full = [q_pick_traj.value(t).flatten() for t in np.linspace(0, t_pick_end, 50)]
    q_full += [PRETHROW_JA] + q_throw_samples[1:]
    
    q_full_traj = PiecewisePolynomial.CubicShapePreserving(t_full, np.array(q_full).T)
    
    # Gripper trajectory
    t_grasp_close = t3
    t_release = t_pick_end + T_GO_TO_PRETHROW + RELEASE_FRAC * THROW_DURATION
    
    g_full_traj = PiecewisePolynomial.FirstOrderHold(
        [0, t_grasp_close, t_grasp_close + 0.01, t_release, t_release + 0.01, q_full_traj.end_time()],
        np.array([[GRIPPER_OPEN, GRIPPER_OPEN, GRIPPER_CLOSED, GRIPPER_CLOSED, GRIPPER_OPEN, GRIPPER_OPEN]])
    )
    
    print(f"  Trajectory duration: {q_full_traj.end_time():.2f}s")
    print(f"  Release at: {t_release:.2f}s")
    
    return q_full_traj, g_full_traj, t_release, q_full_traj.end_time()


# Create trajectory for RED ball only
print("Creating RED ball trajectory (throw toward red bin at +X)...")
q_red_traj, g_red_traj, t_red_release, t_red_end = create_pick_throw_trajectory(
    detected_red_pos, RED_THROW_ANGLE, "RED"
)

print("\n" + "="*60)
print(f"RED BALL TRAJECTORY READY")
print(f"  Throw angle: {np.degrees(RED_THROW_ANGLE):.1f}°")
print(f"  Duration: {t_red_end:.2f}s")
print("="*60)

=== TRAJECTORY PLANNING (Red Ball) ===
Creating RED ball trajectory (throw toward red bin at +X)...

--- Planning RED trajectory ---
  Ball position: [ 2.24477300e-04 -4.99852757e-01  5.00000000e-02]
  Throw angle: -15.0 deg
  Trajectory duration: 4.40s
  Release at: 4.20s

RED BALL TRAJECTORY READY
  Throw angle: -15.0°
  Duration: 4.40s


In [24]:
# EXECUTE RED BALL: Pick + Throw simulation

print("=== RED BALL SIMULATION ===\n")

meshcat.Delete()

# Build scene 
builder_red = DiagramBuilder()
scenario_red = LoadScenario(data=scenario_yaml)
station_red = MakeHardwareStation(scenario_red, meshcat=meshcat)
builder_red.AddSystem(station_red)

# Add cameras for perception after throw
plant_red = station_red.GetSubsystemByName("plant")
scene_graph_red = station_red.GetSubsystemByName("scene_graph")

# Add renderer
renderer_name_red = "renderer_red"
scene_graph_red.AddRenderer(renderer_name_red, MakeRenderEngineVtk(RenderEngineVtkParams()))

# Add 4 cameras (same positions as setup)
camera_config_red = CameraConfig()
camera_config_red.width = 640
camera_config_red.height = 480
camera_config_red.fps = 10.0
camera_config_red.renderer_name = renderer_name_red

camera_sensors_red = []
camera_infos_red = []

for i, X_cam in enumerate(camera_transforms):
    color_cam, depth_cam = camera_config_red.MakeCameras()
    sensor = builder_red.AddSystem(RgbdSensor(
        parent_id=scene_graph_red.world_frame_id(),
        X_PB=X_cam,
        color_camera=color_cam,
        depth_camera=depth_cam
    ))
    builder_red.Connect(station_red.GetOutputPort("query_object"), sensor.query_object_input_port())
    camera_sensors_red.append(sensor)
    camera_infos_red.append(color_cam.core().intrinsics())

# Connect trajectory sources
q_source_red = builder_red.AddSystem(TrajectorySource(q_red_traj))
g_source_red = builder_red.AddSystem(TrajectorySource(g_red_traj))

builder_red.Connect(q_source_red.get_output_port(), station_red.GetInputPort("iiwa.position"))
builder_red.Connect(g_source_red.get_output_port(), station_red.GetInputPort("wsg.position"))

diagram_red = builder_red.Build()

# Get ball bodies
red_ball_body = plant_red.GetBodyByName("ball", plant_red.GetModelInstanceByName("red_ball"))
blue_ball_body = plant_red.GetBodyByName("ball", plant_red.GetModelInstanceByName("blue_ball"))

# Run simulation
simulator_red = Simulator(diagram_red)
simulator_red.set_target_realtime_rate(1.0)

print(f"Running red ball pick + throw...")
print(f"  Red bin at: [{red_bin_pos[0]:.2f}, {red_bin_pos[1]:.2f}]")

sim_context_red = simulator_red.get_mutable_context()

meshcat.StartRecording()

# Run until red throw is complete + settling time
t_end_red = q_red_traj.end_time() + 2.0

while sim_context_red.get_time() < t_end_red:
    simulator_red.AdvanceTo(sim_context_red.get_time() + 0.02)

meshcat.StopRecording()
meshcat.PublishRecording()

# Check red ball result
plant_context_red = plant_red.GetMyContextFromRoot(sim_context_red)
final_red_pos = plant_red.EvalBodyPoseInWorld(plant_context_red, red_ball_body).translation()
final_blue_pos = plant_red.EvalBodyPoseInWorld(plant_context_red, blue_ball_body).translation()

red_dist = np.sqrt((final_red_pos[0] - red_bin_pos[0])**2 + (final_red_pos[1] - red_bin_pos[1])**2)
red_in_bin = red_dist < 0.15

print(f"\n--- Red Ball Result ---")
print(f"  Final position: [{final_red_pos[0]:.3f}, {final_red_pos[1]:.3f}, {final_red_pos[2]:.3f}]")
print(f"  Distance to red bin: {red_dist*100:.1f} cm")
print(f"  In bin: {'YES' if red_in_bin else 'NO'}")

print(f"\n--- Blue Ball Status ---")
print(f"  Current position: [{final_blue_pos[0]:.3f}, {final_blue_pos[1]:.3f}, {final_blue_pos[2]:.3f}]")

print("\n" + "="*60)
print("RED THROW COMPLETE - Ready for blue ball perception")
print("="*60)

=== RED BALL SIMULATION ===



Running red ball pick + throw...
  Red bin at: [0.35, 1.65]

--- Red Ball Result ---
  Final position: [0.270, 1.630, 0.070]
  Distance to red bin: 8.2 cm
  In bin: YES

--- Blue Ball Status ---
  Current position: [-0.150, -0.550, 0.050]

RED THROW COMPLETE - Ready for blue ball perception


In [25]:
# PERCEPTION #2: Find blue ball using cameras (after red throw)

print("=== BLUE BALL PERCEPTION (Post Red Throw) ===\n")

from scipy.optimize import least_squares

def fit_sphere_ls(points, known_radius):
    """Fit sphere center given surface points and known radius."""
    def residuals(center, pts, r):
        return np.linalg.norm(pts - center, axis=1) - r
    result = least_squares(residuals, np.mean(points, axis=0), args=(points, known_radius))
    return result.x

# Use cameras from the current simulation to find blue ball
all_points_world_blue = []
all_colors_blue = []

for cam_idx, (sensor, cam_info, X_cam) in enumerate(zip(camera_sensors_red, camera_infos_red, camera_transforms), 1):
    print(f"--- Camera {cam_idx} ---")
    
    sensor_context = sensor.GetMyContextFromRoot(sim_context_red)
    
    color_image = sensor.color_image_output_port().Eval(sensor_context)
    rgb_img = color_image.data[:, :, :3].astype(np.uint8)
    
    depth_image = sensor.depth_image_32F_output_port().Eval(sensor_context)
    depth_img = depth_image.data.squeeze().astype(np.float32)
    
    points_camera, colors = depth_image_to_point_cloud(depth_img, rgb_img, cam_info)
    print(f"  Generated {len(points_camera)} points")
    
    points_world = transform_points_to_world(points_camera, X_cam)
    
    all_points_world_blue.append(points_world)
    all_colors_blue.append(colors)

merged_points_blue = np.vstack(all_points_world_blue)
merged_colors_blue = np.vstack(all_colors_blue)

print(f"\n✓ Merged {len(merged_points_blue)} points from {len(camera_sensors_red)} cameras\n")

# Find blue ball
print("--- Detecting Blue Ball ---")

above_table_mask = merged_points_blue[:, 2] > 0.02
above_table_points = merged_points_blue[above_table_mask]
above_table_colors = merged_colors_blue[above_table_mask]
print(f"Points above table: {len(above_table_points)}")

# Blue detection
blue_score = above_table_colors[:, 2] - np.maximum(above_table_colors[:, 0], above_table_colors[:, 1])

num_blue_points = int(max(50, int(len(above_table_points) * 0.02)))
blue_threshold = np.sort(blue_score)[-num_blue_points]

is_blue = blue_score >= blue_threshold
blue_points = above_table_points[is_blue].copy()
print(f"Bluest {num_blue_points} points selected")

# Clustering
detected_blue_pos_new = None
if len(blue_points) > 10:
    centroid = np.median(blue_points, axis=0)
    
    cluster_radius = 0.06
    for iteration in range(3):
        distances = np.linalg.norm(blue_points - centroid, axis=1)
        inliers = distances < cluster_radius
        if np.sum(inliers) > 10:
            centroid = np.median(blue_points[inliers], axis=0)
            blue_points = blue_points[inliers]
            cluster_radius = 0.05
    
    # FIT SPHERE using least squares
    fitted_center = fit_sphere_ls(blue_points, ball_radius)
    detected_blue_pos_new = np.array([fitted_center[0], fitted_center[1], ball_radius])
    
    print(f"\nDetected blue ball at: {detected_blue_pos_new}")
    print(f"Ground truth (current): {final_blue_pos}")
    error_blue = np.linalg.norm(detected_blue_pos_new[:2] - final_blue_pos[:2])
    print(f"Error: {error_blue*1000:.1f}mm")
    print(f"Cluster size: {len(blue_points)} points")
    
    # Visualize
    meshcat.SetObject("perception/detected_blue_new", Sphere(0.055), Rgba(0.0, 1.0, 1.0, 0.6))
    meshcat.SetTransform("perception/detected_blue_new", RigidTransform(detected_blue_pos_new))
    
    for i, pt in enumerate(blue_points[::3]):
        meshcat.SetObject(f"perception/blue_pts_new/pt_{i}", Sphere(0.008), Rgba(0.0, 0.0, 1.0, 0.9))
        meshcat.SetTransform(f"perception/blue_pts_new/pt_{i}", RigidTransform(pt))
else:
    print("ERROR: Could not detect blue ball!")

print("\n" + "="*60)
if detected_blue_pos_new is not None:
    print(f"✓ BLUE BALL DETECTED at [{detected_blue_pos_new[0]:.3f}, {detected_blue_pos_new[1]:.3f}]")
    print(f"  Error: {error_blue*1000:.1f}mm")
else:
    print("✗ BLUE BALL NOT DETECTED")
print("="*60)

=== BLUE BALL PERCEPTION (Post Red Throw) ===

--- Camera 1 ---
  Generated 299155 points
--- Camera 2 ---
  Generated 250691 points
--- Camera 3 ---
  Generated 287167 points
--- Camera 4 ---
  Generated 271444 points

✓ Merged 1108457 points from 4 cameras

--- Detecting Blue Ball ---
Points above table: 58292
Bluest 1165 points selected

Detected blue ball at: [-0.14974575 -0.55002682  0.05      ]
Ground truth (current): [-0.15       -0.55        0.04999804]
Error: 0.3mm
Cluster size: 1178 points

✓ BLUE BALL DETECTED at [-0.150, -0.550]
  Error: 0.3mm


In [26]:
# PLAN BLUE BALL TRAJECTORY (using fresh perception)

if detected_blue_pos_new is None:
    raise RuntimeError("Cannot plan trajectory - blue ball not detected")

print("=== TRAJECTORY PLANNING (Blue Ball) ===")

# Get current gripper pose from simulation as starting point
gripper_body = plant_red.GetBodyByName("body")
X_WG_current = plant_red.EvalBodyPoseInWorld(plant_context_red, gripper_body)
print(f"Current gripper position: {X_WG_current.translation()}")

# Create blue ball trajectory using the FRESH detected position
print(f"\nCreating BLUE ball trajectory (throw toward blue bin at -X)...")
print(f"  Using detected position: {detected_blue_pos_new}")

q_blue_traj, g_blue_traj, t_blue_release, t_blue_end = create_pick_throw_trajectory(
    detected_blue_pos_new, BLUE_THROW_ANGLE, "BLUE"
)

print("\n" + "="*60)
print(f"BLUE BALL TRAJECTORY READY")
print(f"  Throw angle: {np.degrees(BLUE_THROW_ANGLE):.1f}°")
print(f"  Duration: {t_blue_end:.2f}s")
print("="*60)

=== TRAJECTORY PLANNING (Blue Ball) ===
Current gripper position: [0.07072307 0.26883739 1.2266632 ]

Creating BLUE ball trajectory (throw toward blue bin at -X)...
  Using detected position: [-0.14974575 -0.55002682  0.05      ]

--- Planning BLUE trajectory ---
  Ball position: [-0.14974575 -0.55002682  0.05      ]
  Throw angle: 15.0 deg
  Trajectory duration: 4.40s
  Release at: 4.20s

BLUE BALL TRAJECTORY READY
  Throw angle: 15.0°
  Duration: 4.40s


In [27]:
# EXECUTE BLUE BALL: Pick + Throw simulation
# Continues from current state (red ball already thrown)

print("=== BLUE BALL SIMULATION ===\n")

# We need to rebuild the scene but with the red ball already in the bin
# For simplicity, rebuild fresh but only execute blue trajectory

meshcat.Delete()

# Build new scene
builder_blue = DiagramBuilder()
scenario_blue = LoadScenario(data=scenario_yaml)
station_blue = MakeHardwareStation(scenario_blue, meshcat=meshcat)
builder_blue.AddSystem(station_blue)

plant_blue = station_blue.GetSubsystemByName("plant")

# Connect blue trajectory
q_source_blue = builder_blue.AddSystem(TrajectorySource(q_blue_traj))
g_source_blue = builder_blue.AddSystem(TrajectorySource(g_blue_traj))

builder_blue.Connect(q_source_blue.get_output_port(), station_blue.GetInputPort("iiwa.position"))
builder_blue.Connect(g_source_blue.get_output_port(), station_blue.GetInputPort("wsg.position"))

diagram_blue = builder_blue.Build()

# Get blue ball body
blue_ball_body_sim = plant_blue.GetBodyByName("ball", plant_blue.GetModelInstanceByName("blue_ball"))

# Run simulation
simulator_blue = Simulator(diagram_blue)
simulator_blue.set_target_realtime_rate(1.0)

print(f"Running blue ball pick + throw...")
print(f"  Blue bin at: [{blue_bin_pos[0]:.2f}, {blue_bin_pos[1]:.2f}]")
print(f"  Target ball position: {detected_blue_pos_new}")

sim_context_blue = simulator_blue.get_mutable_context()

meshcat.StartRecording()

t_end_blue = q_blue_traj.end_time() + 2.0

while sim_context_blue.get_time() < t_end_blue:
    simulator_blue.AdvanceTo(sim_context_blue.get_time() + 0.02)

meshcat.StopRecording()
meshcat.PublishRecording()

# Check result
plant_context_blue = plant_blue.GetMyContextFromRoot(sim_context_blue)
final_blue_pos_sim = plant_blue.EvalBodyPoseInWorld(plant_context_blue, blue_ball_body_sim).translation()

blue_dist = np.sqrt((final_blue_pos_sim[0] - blue_bin_pos[0])**2 + (final_blue_pos_sim[1] - blue_bin_pos[1])**2)
blue_in_bin = blue_dist < 0.15

print(f"\n--- Blue Ball Result ---")
print(f"  Final position: [{final_blue_pos_sim[0]:.3f}, {final_blue_pos_sim[1]:.3f}, {final_blue_pos_sim[2]:.3f}]")
print(f"  Distance to blue bin: {blue_dist*100:.1f} cm")
print(f"  In bin: {'YES' if blue_in_bin else 'NO'}")

print("\n" + "="*60)
print("SORTING COMPLETE!")
print(f"  Red ball in red bin:   {'YES' if red_in_bin else 'NO'}")
print(f"  Blue ball in blue bin: {'YES' if blue_in_bin else 'NO'}")
if red_in_bin and blue_in_bin:
    print("\n  PERFECT SORT!")
print("="*60)

=== BLUE BALL SIMULATION ===

Running blue ball pick + throw...
  Blue bin at: [-0.35, 1.65]
  Target ball position: [-0.14974575 -0.55002682  0.05      ]

--- Blue Ball Result ---
  Final position: [-0.283, 1.570, 0.070]
  Distance to blue bin: 10.4 cm
  In bin: YES

SORTING COMPLETE!
  Red ball in red bin:   YES
  Blue ball in blue bin: YES

  PERFECT SORT!
